In [1]:
import pandas as pd
import numpy as np
import pycountry

In [2]:
##LOAD MAIN DF


main_df = pd.read_csv("main_df_csv_backup.csv")

In [3]:
main_df.index = main_df.index+1

In [4]:
main_df

,Volcano Name,Eruption Type,Start Date,Max. VEI,Country,Volcanic_Region,Volcano_type,Latitude,Longitude,Summit
1,Ambae,Confirmed Eruption,2021 Dec 5,--,Vanuatu,Melanesia and Australia,Shield,15.389°S,167.835°E,1496.0
2,Pinatubo,Confirmed Eruption,2021 Nov 30,--,Philippines,Philippines and SE Asia,Stratovolcano,15.13°N,120.35°E,1486.0
3,Iliwerung,Confirmed Eruption,2021 Nov 28,--,Indonesia,Indonesia,Complex,8.532°S,123.573°E,583.0
4,Taal,Confirmed Eruption,2021 Nov 15,--,Philippines,Philippines and SE Asia,Caldera,14.002°N,120.993°E,311.0
5,Turrialba,Confirmed Eruption,2021 Nov 3,--,Costa Rica,México and Central America,Stratovolcano,10.025°N,83.767°W,3340.0
...,...,...,...,...,...,...,...,...,...,...
11218,"Yojoa, Lago",Confirmed Eruption,1073 BCE,--,Honduras,México and Central America,Volcanic field,14.964°N,87.983°W,1060.0
11219,Quetrupillan,Confirmed Eruption,1345 BCE,3,Chile,South America,Stratovolcano,39.496°S,71.722°W,2360.0
11220,Saunders,Confirmed Eruption,NaN,0,United Kingdom,Antarctica,Stratovolcano,57.8°S,26.483°W,843.0
11221,"Fournaise, Piton de la",Confirmed Eruption,NaN,--,France,Middle East and Indian Ocean,Shield,21.244°S,55.708°E,2632.0


In [25]:
main_df.describe(include = "all")

,Volcano Name,Max. VEI,Country,Volcanic_Region,Volcano_type,Summit,Year,Latitude_value,Longitude_value
count,7675.000000,7675.000000,7675.000000,7675.000000,7675.000000,7675.000000,7675.000000,7675.000000,7675.000000
mean,185.957655,2.011987,12.315309,7.470358,2.314397,2154.024235,1161.183322,13.810667,39.804460
std,167.368447,1.154525,11.531268,4.976829,2.692385,1392.650330,1975.468627,30.016853,114.253192
min,1.000000,0.000000,1.000000,1.000000,1.000000,-4200.000000,-9650.000000,-77.530000,-179.970000
25%,50.000000,1.000000,4.000000,3.000000,1.000000,1241.000000,1659.000000,-7.540000,-75.330000
50%,130.000000,2.000000,10.000000,7.000000,1.000000,1916.000000,1894.000000,13.257000,105.423000
75%,274.000000,2.000000,18.000000,10.000000,3.000000,2910.000000,1965.000000,37.748000,139.394000
max,687.000000,7.000000,70.000000,19.000000,19.000000,6739.000000,2020.000000,85.608000,179.580000


In [26]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7675 entries, 1 to 7675
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Volcano Name     7675 non-null   int64  
 1   Max. VEI         7675 non-null   int16  
 2   Country          7675 non-null   int64  
 3   Volcanic_Region  7675 non-null   int64  
 4   Volcano_type     7675 non-null   int64  
 5   Summit           7675 non-null   float64
 6   Year             7675 non-null   int32  
 7   Latitude_value   7675 non-null   float64
 8   Longitude_value  7675 non-null   float64
dtypes: float64(3), int16(1), int32(1), int64(4)
memory usage: 464.8 KB


In [7]:
main_df.dropna(inplace = True)

In [8]:
main_df[['Year', 'BCE']] = main_df['Start Date'].str.split(" ",n=1,expand=True)#split the data with spaces
main_df['Year'] = main_df['Year'].astype(int) #casting
main_df['BCE'].fillna(value = 'de', inplace = True)#filling nan vallues for the run

In [9]:
for index in range(11141):
    if main_df.iloc[index, 11] == "BCE":                  #if BCE turn the year to -1
        main_df.iloc[index, 10]=main_df.iloc[index, 10]*-1
main_df = main_df.drop(['BCE', 'Start Date'],axis=1) #drop the unecessary columns

In [10]:
main_df.drop(main_df.loc[main_df["Eruption Type"] != 'Confirmed Eruption'].index, inplace=True)

In [11]:
main_df = main_df.drop(['Eruption Type'],axis=1)

In [12]:
main_df['Max. VEI'] = main_df['Max. VEI'].replace(['--'],'999')
main_df['Max. VEI'] = pd.to_numeric(main_df['Max. VEI'],downcast= 'integer')#cast VEI from string to int
main_df.drop(main_df.loc[main_df["Max. VEI"] == 999].index, inplace=True,axis=0)

In [13]:
main_df[['Latitude_value', 'direction']] = main_df['Latitude'].str.split("°",n=1,expand=True)#split the data with spaces
###
# N -> +
# S -> -
# in that order well be fixing the lat and long (also work in google maps)
main_df['Latitude_value'] = main_df['Latitude_value'].astype(float)
for index in range(main_df.shape[0]):
    if main_df.iloc[index, 10] == "S":#if S turn the lat to -1
        main_df.iloc[index, 9] *=  -1
main_df = main_df.drop(['direction', 'Latitude'],axis=1) #drop the unecessary columns

In [14]:
main_df[['Longitude_value', 'direction']] = main_df['Longitude'].str.split("°",n=1,expand=True)#split the data with spaces
###
# E -> +
# W -> -
# in that order well be fixing the lat and long (also work in google maps)
main_df['Longitude_value'] = main_df['Longitude_value'].astype(float)
for index in range(main_df.shape[0]):
    if main_df.iloc[index, 10] == "W":#if S turn the lat to -1
        main_df.iloc[index, 9] *=  -1
main_df = main_df.drop(['direction', 'Longitude'],axis=1) #drop the unecessary columns

In [15]:
#DO NOT DELETE
#correlations=[]
#arr=[]
#cols1=main_df.columns
#cols2=main_df.columns
#i=0
#for col1 in cols1:
#    j=0
#    for col2 in cols2:
#        if col1==col2:
#            break
#         if main_df[col1].corr(main_df[col2])>=0.5 or main_df[col1].main_df[col1].corr(df[col2])>=0.5:
#             arr.append(f"({main_df.columns[i]}, {main_df.columns[j]})")
#         j+=1
#     i+=1
# print(arr)

In [16]:
main_df["Volcano_type"] = main_df["Volcano_type"].replace("\((\w+)\)", "",regex = True)#Regular Expressions - ze til!
main_df["Volcano_type"].replace('Stratovolcano?', 'Stratovolcano',inplace =True)
volcano_type_arr=main_df["Volcano_type"].unique()
volcano_type_list =volcano_type_arr.tolist()
volcano_type_list
volcano_type_dict = {i:j for i,j in zip(volcano_type_list,range(1,80))} # i is key, j value, i iterate on countrylist, j on range
main_df["Volcano_type"].replace(volcano_type_dict,inplace = True)

In [17]:
country_arr=main_df["Country"].unique()
country_list =country_arr.tolist()
country_list
countrydict = {i:j for i,j in zip(country_list,range(1,81))} # i is key, j value, i iterate on countrylist, j on range
main_df["Country"].replace(countrydict,inplace = True)

In [18]:
volc_name_arr=main_df["Volcano Name"].unique()
volc_name_list =volc_name_arr.tolist()
volc_name_list
volc_name_dict = {i:j for i,j in zip(volc_name_list,range(1,857))} # i is key, j value, i iterate on countrylist, j on range
main_df["Volcano Name"].replace(volc_name_dict,inplace = True)

In [19]:
volc_reg_arr=main_df["Volcanic_Region"].unique()
volc_reg_list =volc_reg_arr.tolist()
volc_reg_list
volc_reg_dict = {i:j for i,j in zip(volc_reg_list,range(1,100))} # i is key, j value, i iterate on countrylist, j on range
main_df["Volcanic_Region"].replace(volc_reg_dict,inplace = True)

In [20]:
main_df.reset_index(inplace = True,drop=True)
main_df.index += 1

In [22]:
main_df

,Volcano Name,Max. VEI,Country,Volcanic_Region,Volcano_type,Summit,Year,Latitude_value,Longitude_value
1,1,1,1,1,1,2910.0,2020,-7.540,110.446
2,2,4,2,2,1,1220.0,2020,13.330,-61.180
3,3,3,3,3,1,1745.0,2020,12.702,-87.004
4,4,1,1,1,1,2329.0,2020,-7.942,112.950
5,5,0,4,4,2,1222.0,2020,19.421,-155.287
...,...,...,...,...,...,...,...,...,...
7671,113,5,14,13,1,1978.0,-9650,-39.157,175.632
7672,601,0,4,19,12,2005.0,-60,43.420,-113.500
7673,687,1,23,14,12,1146.0,-450,-4.889,31.933
7674,586,3,19,8,1,2360.0,-658,-39.496,-71.722


In [24]:
####SAVE DF ###

main_df.to_csv("final-df.csv",index = False)